Pulse-based Experiments with QubiC
==============================

Prerequisites: this tutorial assumes that the QubiC [software](https://gitlab.com/LBL-QubiC/software) and [distributed processor](https://gitlab.com/LBL-QubiC/distributed_processor) have been installed (branch ``rfsoc`` if installing from the source repo).
Additionally, ``numpy`` and ``bqplot`` are required for displaying simulated DAC output.

---
**NOTE**

The simple characterization experiments described here are all available as pre-defined, highly optimized, routines in QubiC.
The step-by-step approach used is for educational purposes only.
For actual work, [these pre-defined routines](https://gitlab.com/LBL-QubiC/experiments/chipcalibration/-/tree/rfsoc/chipcalibration?ref_type=heads) should be used.
---

In [1]:
%load_ext autoreload
%autoreload 2

Introduction
---------------

QubiC is capable of playing arbitrary pulses for qubit drive and readout, allowing you to get the most out of your quantum hardware by controlling the precise timing and dynamics.
It comes with APIs that give you full control, but resource efficiency is key to scaling and quick experiment turnaround.
The goal of this notebook is therefore to show how to efficiently construct a pulse-based experiment in QubiC.

### Pulse Nomenclature

<table style="padding: 0px"><tr></tr><tr>
<td style="width: 50%; padding: 0px">
<div align="left" style="text-align: left; font-size: 120%">
<p>A <i>pulse</i> is series of output voltages at specific time stamps.
It is constructed by first defining a pulse <i>envelope</i> at regular steps called <i>samples</i>.
For example, an envelope could take the form of a Gaussian shape, with at each sample the value corresponding to a Gaussian function.
Samples become time stamps by specifying to QubiC the <i>sample rate</i> and <i>interpolation ratio</i> to get the number of samples played per second.</p>
<p>The qubit oscillates, however, so to ensure that it actually "sees" the envelope as intended, the signal needs to be modulated with a <i>carrier signal</i> to transform in from the laboratory frame of reference into the qubit one.
Finally, the scale of the pulse is determined by specifying a pulse <i>amplitude</i>.</p>
</div></td>
<td style="width: 50%">
  <img src='./images/Illustration_of_Amplitude_Modulation.png'>
</td>
</tr>
<tr></tr><tr><td></td>
<td><div style="font-size: 70%">Source: Wikipedia</div></td>
</table>
A pulse envelope can be constructed directly and provided to QubiC in the form of a <span style="font-family:monospace">numpy</span> array, exactly specifying the output value for each sample.
However, it is much more efficient to provide parametrizations of pre-defined envelopes, such as those from QubiC's <i>pulse library</i> of commonly used ones.
Such a parametrized envelope can be reused, e.g. different qubits can share the same envelope shape but different carrier frequencies and amplitudes, thus reducing the overall memory requirements of the system, which is essential to achieve scale.

### Pulse Scheduling

QubiC allows you to specify precisely when to start playing a pulse, using a start time counting from the beginning of the experiment.
It uses human-readable, symbolic labels to specify the channel on which the pulse should be played.
For example, "Q1.qdrv" refers to the qubit drive channel of qubit 1.
A separate <a href="https://gitlab.com/LBL-QubiC/software/-/wikis/Understanding-Channel-Configuration"><span style="font-family:monospace">channel_config.json</span></a> file then tells the system to which actual hardware channel this corresponds.
This approach makes pulse programs simpler to understand and keeps them portable.

The pulse start times are typically determined by pragmatic constraints, e.g. a different pulse on another qubit finishing first to reduce cross-talk.
Since the qubit oscillates around the Z-axis, however, such timing in the laboratory frame translates into a phase in the qubit frame of reference.
Thus, to have full control over the timing in the qubit frame, QubiC allows you to specify the <i>phase</i> of the carrier signal for the pulse at each place of use in the experiment.

### Standard Tooling

The following set of python modules are necessary for running QubiC.
If importing is successful, your system is all setup!

In [2]:
# standard QubiC modules
import qubic.toolchain as tc
import qubic.rpc_client as rc
import qubitconfig.qchip as qc
from distproc.hwconfig import FPGAConfig, load_channel_configs

# useful Python modules
import numpy as np
import bqplot as plt

Next, we load the necessary configurations, both the hardware setup and qubit calibration data.
These files are included alongside this tutorial and should be tuned to match your actual QubiC setup.

In [3]:
# FPGA timing information for the scheduler
fpga_config = FPGAConfig(**{'fpga_clk_period': 2.e-9, 
                            'alu_instr_clks': 5, 
                            'jump_cond_clks': 5, 
                            'jump_fproc_clks': 5, 
                            'pulse_regwrite_clks': 3})

# mapping of programmatic labels to actual channels
channel_configs = load_channel_configs('channel_config.json')

# gate definitions and qubit calibration data
qchip = qc.QChip('qubitcfg.json')

### Of Pulses and Circuits

QubiC provides APIs at several different levels, from the abstract to the low-level.
The highest, circuit, level makes it a breeze to work with calibrated pulses and is also very human-readable.
It provides operations for scheduling (delays) and synchronizatin (barriers)
The circuit level is transformed to the compiled level, by taking the information from the configuration files we loaded above.
The result, the compiled level, is very programmable and particularly useful if e.g. the pulses you design come from a different software stack, especially if that software already provides its own scheduling as at this level, time is absolute from a fixed start point.
Finally, the compiled level gets assembled: the actual memory to copy over to the FPGA and the command words for QubiC to execute.
This is particularly of use if you are interested in development of QubiC itself.

In all cases, the pulse descriptions are Python dictionaries with conventional labels as keys to the parameters of the pulse. Below are some annotated examples at the highest level.

In [4]:
# This is the highest level description and refers to a calibrated
# pulse in the "qubitcfg.json" file. Specifically, "X90Q1" under
# the "Gates" section: a pi/2 pulse played on qubit 1.
X90_Q1 = {'name': 'X90',     # label of the gate referenced
          'qubit': 'Q1'      # the qubit that it applies to
}

# This is the same pulse as above, but rather than looking up all
# entries in "qubitcfg.json", they are explicitly listed here. This
# way, these entries can be programmatically updated in Python.
X90_Q1 = {'name': 'pulse',                     # generic pulse
          'phase': 0,                          # phase starts at 0
          'freq': 4675138775,                  # carrier frequence in Hz
          'amp': 0.48150320341813146,          # amplitude applied to envelope
          'twidth': 2.4e-08,                   # duration in seconds
          'env': {                             # the pulse envelope
              'env_func': 'cos_edge_square',   # function describing the envelope
              'paradict': {                    # parameters input to env_func
                  'ramp_fraction': 0.25        # fraction of the square to be
              }                                #  smoothed by the cosine
          },
          'dest': 'Q1.qdrv'                    # channel to play the pulse on, here
                                               #  the drive of qubit 1
}


A QubiC circuit, or program, is a collection of pulses (drive and readout) and (optional) scheduling operations. Below is an example of a program: a bit flip on qubit 1, followed by a measurement.

In [5]:
# A delay parameter, the purpose of which will be explained later
param_delay = {'name': 'delay', 't': 100.e-9}

circuit = [
    # Passive reset (this assumes the qubit T1 is roughy 100us.
    {'name': 'delay', 't': 500.e-6}, 
    
    # Our X90 pulse on Q1, defined above.
    X90_Q1,
    
    # Another X90, to achieve an X180, ie. a bit flip.
    X90_Q1,
    
    # An additional delay (see below)
    param_delay,
    
    # A measurement of Q1, defined qubitcfg.json, just like the above
    # X90 was.
    {'name': 'read', 'qubit': 'Q1'},
]

Next, compile and assemble the circuit, for details, see the [Hardware Demo](Hardware Demo Part 2.ipynb) tutorial.
The assembled program can now be send to QubiC and run.
For the purposes of this tutorial, however, we will run a simulator instead: the simulator connects to a server and returns the waveform output as played by QubiC (i.e. how it would look on an oscilloscope), which we can then inspect.

In [6]:
import simulation

In [7]:
simulator = simulation.SimClient()

In [8]:
def simulate(circuit, sim_time=None):
    # compile and assemble circuit
    circuit_compiled  = tc.run_compile_stage(circuit, fpga_config, qchip)
    circuit_assembled = tc.run_assemble_stage(circuit_compiled, channel_configs)

    # use the compiled circuit here for the in-process simulator; use the
    # assembled circuit for the real thing
    circuit = circuit_compiled
    if not hasattr(simulator, 'fake'):
        circuit = circuit_assembled
        if sim_time is None:
            sim_time = 1E-6

    dacout, readout_start = simulator.run_program(circuit, sim_time)

    return dacout, readout_start

def plot_dac(sim_result, qubit=0, lower=None, upper=None, down_sample=3):
    dac_out = sim_result[0][qubit]
    cut = lower is not None or upper is not None
    if lower is None:
        lower = 0
    if upper is None:
        upper = sim_result[1]+3000
    
    if cut:
        dac_out = dac_out[lower:upper]
    
    if down_sample:
        ddac_out = dac_out[::down_sample]

    x_sc = plt.LinearScale()
    y_sc = plt.LinearScale()

    ax_x = plt.Axis(scale=x_sc)
    ax_y = plt.Axis(scale=y_sc, orientation="vertical")

    x_data = np.arange(lower, upper)
    y_data = dac_out
    scatter = plt.Lines(
        x=x_data,
        y=y_data,
        scales={"x": x_sc, "y": y_sc},
    )

    fig = plt.Figure(axes=[ax_x, ax_y], marks=[scatter])
    fig.interaction = plt.PanZoom(scales={"x": [x_sc]})#, "y": [y_sc]})

    return fig

In [9]:
sim_result = simulate(circuit)

In [10]:
fig = plot_dac(sim_result, qubit=1)
fig

Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top':…

### Customizing Pulses



### T1 Experiment

<table style="padding: 0px"><tr></tr><tr>
<td style="width: 50%; padding: 0px">
<div align="left" style="text-align: left; font-size: 120%">
<p>A <a href="https://qiskit.org/ecosystem/experiments/manuals/characterization/t1.html">T1 experiment</a> consists of placing the qubit in the excited, ie. |1>, state followed by a measurement after some delay.
As the delay increases, the number of shots where the qubit will have decayed to the ground, ie. |0>, state increases.
By plotting the ratio of shots with a measurment of the qubit in the excited vs. ground state as a function of the delay, an exponential curve results, which can be fitted to obtain the decay parameter T1 (see figure on the right).</p>
<p>To set this experiment up, we use a series of circuits, each with a different delay before measuring.
If you scroll back up to our first circuit, you should now recognize it as a single step in a series for the T1 measurments.
The first delay is a passive reset, putting the qubit in the |0> state.
Then follow two X90 gates, ie. an X180, putting the qubit in the |1> state.
Next is a parametrized delay and finally, there is a readout.
All that remains is to put the circuit creation in a loop, adjusting the parametrized delay at each step.</p>
</div></td>
<td style="width: 50%">
  <img src='./images/T1.png' width=300>
</td>
</tr>
</table>

In [11]:
import copy

In [12]:
start_delay = 500E-9
delay_step  = 500E-9
n_steps = 20

In [13]:
# note: rerun the cell defining the initial circuit if necessary

t1_circuits = list()
for istep in range(n_steps):
    param_delay['t1'] = start_delay + istep*delay_step
    t1_circuits.append(copy.deepcopy(circuit))